In [3]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime
import os

# === OPTION 1: Manual site IDs ===
# site_ids = ["10987", "3133", "10543"]

# === OPTION 2: Load site IDs from metadata CSV (first 5 STATION_IDs) ===
metadata_path = "meta_data_for_all_active_systems_for_calculation_table_20250401.csv"
metadata_df = pd.read_csv(metadata_path, dtype=str)
site_ids = metadata_df["STATION_ID"].head(5).tolist()

# Create a list to hold summary results
summary_data = []

# Loop through each site ID
for DivrtID in site_ids:
    end_date = datetime.today().strftime("%Y-%m-%d")
    api_url = f"https://www.waterrights.utah.gov/dvrtdb/daily-chart.asp?station_id={DivrtID}&end_date={end_date}&f=json"
    print(f"Fetching data from: {api_url}")

    response = requests.get(api_url)

    if response.status_code == 200:
        data = response.json()

        if "data" in data:
            df = pd.DataFrame(data["data"], columns=["date", "value"])
            df.rename(columns={"date": "Date", "value": "DISCHARGE"}, inplace=True)

            # Convert data types
            df["DISCHARGE"] = pd.to_numeric(df["DISCHARGE"], errors='coerce')
            df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
            df.dropna(subset=["Date", "DISCHARGE"], inplace=True)

            # Filter for valid discharge values
            filtered_discharge = df["DISCHARGE"][(df["DISCHARGE"] > 0) & (~df["DISCHARGE"].isna())]

            # Calculate IQR and upper bound
            Q1 = filtered_discharge.quantile(0.25)
            Q3 = filtered_discharge.quantile(0.75)
            IQR = Q3 - Q1
            upper_bound = Q3 + 1.5 * IQR
            MaxValue_IQR = upper_bound

            # Calculate alternative threshold: 95th percentile
            MaxValue_95Perc = filtered_discharge.quantile(0.95)

            # Choose final max threshold: min of IQR-based upper bound and 95th percentile
            MaxValueFinal = min(MaxValue_IQR, MaxValue_95Perc)

            # Rate of change calculations
            df["RoC"] = df["DISCHARGE"].diff().replace([np.inf, -np.inf], np.nan)
            roc_values = df["RoC"].dropna().abs()
            Average_Rate_Of_Change = roc_values.mean()
            Roc_90th_Percentile = roc_values.quantile(0.90)
            Roc_Standard_Deviation = roc_values.std()

            # Discharge standard deviation
            Discharge_Standard_Deviation = df["DISCHARGE"].std()

            # Calculate Unusual Spike as the min of 90th percentile and std dev
            UnusualChange90thPerc = Roc_90th_Percentile
            UnusualAboveUpperBound = Roc_Standard_Deviation if not np.isnan(Roc_Standard_Deviation) else np.nan
            UnusualSpike = min(UnusualChange90thPerc, UnusualAboveUpperBound) if not np.isnan(UnusualAboveUpperBound) else UnusualChange90thPerc

            # Add results to summary
            summary_data.append({
                "SiteID": DivrtID,
                "Below_Capacity": 0,
                "Over_Capacity": MaxValueFinal,  # Final flagging threshold (min of IQR upper bound and 95th percentile)
                "Min_IQR_Upper_Bound_Value": MaxValue_IQR,
                "Max_Value_95Perc": MaxValue_95Perc,
                "Average_Rate_Of_Change": Average_Rate_Of_Change,
                "Unusual_Change_90th_Perc.": UnusualChange90thPerc,
                "Unusual_Above_Upper_Bound": UnusualAboveUpperBound,
                "Unsual_Spike": UnusualSpike,
                "Discharge_Standard_Deviation": Discharge_Standard_Deviation if not np.isnan(Discharge_Standard_Deviation) else np.nan,
            })

        else:
            print(f"Error: 'data' key not found for site {DivrtID}")
    else:
        print(f"Error fetching data for site {DivrtID}: {response.status_code}")

# Export results to CSV
summary_df = pd.DataFrame(summary_data)
summary_df.to_csv("dvrt_data_qualifiers_max_min_RoC_SD_table.csv", index=False)
print("CSV saved as 'dvrt_data_qualifiers_max_min_RoC_SD_table.csv'")

Fetching data from: https://www.waterrights.utah.gov/dvrtdb/daily-chart.asp?station_id=6&end_date=2025-04-18&f=json
Fetching data from: https://www.waterrights.utah.gov/dvrtdb/daily-chart.asp?station_id=18&end_date=2025-04-18&f=json
Fetching data from: https://www.waterrights.utah.gov/dvrtdb/daily-chart.asp?station_id=19&end_date=2025-04-18&f=json
Fetching data from: https://www.waterrights.utah.gov/dvrtdb/daily-chart.asp?station_id=21&end_date=2025-04-18&f=json
Fetching data from: https://www.waterrights.utah.gov/dvrtdb/daily-chart.asp?station_id=26&end_date=2025-04-18&f=json
CSV saved as 'dvrt_data_qualifiers_max_min_RoC_SD_table.csv'
